In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import metpy
import sys
import pandas as pd
import netCDF4 as nc
from os import walk
import os
import metpy.calc as mpcalc
from metpy.units import units
#from mpl_toolkits.basemap import Basemap
import cartopy.crs as ccrs
#import gcm_filters
import copy
import datetime
import calendar
from windspharm.standard import VectorWind
import math
from scipy import ndimage
from scipy import fft



def wn_filter_3d(y, kmin, kmax):
    ############################################################################################################################
    # - Wavenumber restriction: Take a 3-D function and return it restricted to the (kmin,kmax) wavenumber range. 
    # Author: Sandro W. Lubis (Mar 2023, PNNL)
    ############################################################################################################################
    # - INPUT:
    # * y: variable with (time,lat,lon) dimensions
    # * kmin, kmax: wavenumber range
    ###########################################################################################################################
    ffty = fft.fft(y.values,axis=-1)
    mask = np.zeros((ffty.shape))
    mask[:,:,kmin:kmax+1] = 1 # Keep certain freqs only. Values outside the selected range remain zero. 
    mafft = ffty*mask
    fedit = fft.ifft(mafft)
    fedit = 2*fedit.real # Since the ignored negative frequencies would contribute the same as the positive ones.
    #  the DC component of the signal is unique and should not be multiplied by 2:
    if kmin == 0:
        fedit = fedit - ffty.real[0]/(ffty.shape[-1]) # Subtract the pre-edited function's mean. We don't want a double contribution from it in fedit (as was demanded by fedit=2*fedit.real).
    elif kmin > 0:
        fedit = fedit + ffty.real[0]/(ffty.shape[-1])  # Add the pre-edited function's mean. The zero frequency in FFT should never be left out when editing a function. 
    fedit = y.copy(data=fedit,deep=False)
    return fedit


def wn_range(LAT,wavelength):
    #calculates a kmin and kmax to pass into wn_filter_3d
    ###### input
    #lat in degrees 
    #wavelength (full wavelength panning max and min) - in KILOMETERS
    
    Rearth = 6378  # earth radius in km
    ktarget = ( 2 * math.pi * Rearth * math.cos( (math.pi/180)* LAT ) ) / wavelength
    return ktarget

########################################################################
# directories containing 3D and 2D ERA5 reanalysis files: 
########################################################################

#2d era5 analysis file locations:
root2Dvars = '/<YOUR DIRECTORY HERE>/'
root2Dvint = '/<YOUR DIRECTORY HERE>/'

#3d era5 analysis file locations: 
root3Dvars = '/<YOUR DIRECTORY HERE>/'
rootPVdir = '/<YOUR DIRECTORY HERE>/'

# output directory and filenames:
fileout = '/<YOUR DIRECTORY HERE>/ERA5_bandpass_wl' + str(WL_min) + 'to' + str(WL_max) +'km_perlat_vort_sf_b'
            

WL_min = 500   #shortest desired post-fft-filter wavelength (km)
WL_max = 2500  #longest desired post-fft-filter wavelength (km)


########################################################################
# list of all YYYYMM dirs for which there are 2D and 3D ERA5 files on nersc (separetely), 
# then find common dates
########################################################################


# find all ERA5 data months that we have files for in root2Dvars (months_var2d):
months_var2d = []
for (dirpath, dirnames, filenames) in walk(root2Dvars):
    months_var2d.extend(dirnames)
    break
months_var2d = [int(x) for x in months_var2d]
months_var2d.sort()


# find all ERA5 data months that we have files for in root3Dvars (dates_var3d):
months_var3d = []
for (dirpath, dirnames, filenames) in walk(root3Dvars):
    months_var3d.extend(dirnames)
    break
months_var3d = [int(x) for x in months_var3d]
months_var3d.sort()


# find common dates among the 2D and 3D variable list (commondates):   
list1_as_set = set(months_var2d)
intersection = list1_as_set.intersection(months_var3d)
commonmonths = list(intersection)
commonmonths.sort()

# Convert common to Year-Month strings for future use (YYYYMM):  
YYYYMM = []
for t in commonmonths:
    yyyy = str(t)[0:4]
    mm = str(t)[4:6]
    YYYYMM.append(yyyy + '-' + mm)
    
  
    
#########################################################################################################
# generate file lists of the variables that we want from the list of common months   
#########################################################################################################

# list of all 2D-metric files of a particular variable (e.g., tcwv) in the whole climo period (PWfiles):  
PWfiles = []
for date in commonmonths:
    pwdatepath = root2Dvars + str(date)+ "/"
    # print(pwdatepath)
    for r, d, f in walk(pwdatepath):
        for file in f:
            if 'tcwv' in file:
                PWfiles.append(os.path.join(r,file))
                
VIWVDfiles = []
for date in commonmonths:
    viwvddatepath = root2Dvint + str(date)+ "/"
    # print(pwdatepath)
    for r, d, f in walk(viwvddatepath):
        for file in f:
            if 'viwvd' in file:
                VIWVDfiles.append(os.path.join(r,file))  
        
CAPEfiles = []
for date in commonmonths:
    capedatepath = root2Dvars + str(date)+ "/"
    # print(pwdatepath)
    for r, d, f in walk(capedatepath):
        for file in f:
            if 'cape' in file:
                CAPEfiles.append(os.path.join(r,file))        
                
SPfiles = []
for date in commonmonths:
    spdatepath = root2Dvars + str(date)+ "/"
    # print(pwdatepath)
    for r, d, f in walk(spdatepath):
        for file in f:
            if '_sp.' in file:
                SPfiles.append(os.path.join(r,file))
 

                 
    
# list of all files of a prescribed 3D <variable> in the whole climo period (<VAR>files):  
Ufiles = []
Vfiles = []
Wfiles = []
QVfiles = []
RHfiles = []
Zfiles = []
Tfiles = []
for date in commonmonths:
    datepath = root3Dvars + str(date)+ "/"
    for r, d, f in walk(datepath):
        for file in f:
            if '_u.' in file:
                Ufiles.append(os.path.join(r,file))    
            if '_v.' in file:
                Vfiles.append(os.path.join(r,file)) 
            if '_w.' in file:
                Wfiles.append(os.path.join(r,file))
            if '_q.' in file:
                QVfiles.append(os.path.join(r,file)) 
            if '_r.' in file:
                RHfiles.append(os.path.join(r,file))  
            if '_t.' in file:
                Tfiles.append(os.path.join(r,file))          
            if '_z.' in file:
                Zfiles.append(os.path.join(r,file))
#sort them alpha-numerically:
Ufiles.sort()
Vfiles.sort()
Wfiles.sort()
QVfiles.sort()
RHfiles.sort()    
PWfiles.sort()
Tfiles.sort()
Zfiles.sort()
CAPEfiles.sort()
VIWVDfiles.sort()


# list of all files of a prescribed 3D <variable> in the whole climo period (<VAR>files):  
PVfiles = []
for date in commonmonths:
    datepath = rootPVdir + str(date)+ "/"
    for r, d, f in walk(datepath):
        for file in f:
            if '_pv.' in file:
                PVfiles.append(os.path.join(r,file))
#sort them alpha-numerically:
PVfiles.sort()


# get days in addition to the months (YYYYMMDD_var3d). 
# I'm using the U variable as an example and ASSUMING THAT IF THERE ARE ANY GAPS IN U, THERE ARE EQUIVALENT GAPS IN THE OTHER VARIABLES 

YYYYMMDD_var3d = ["00000000"]  
for file in Ufiles:
    year = str(file)[-24:-20]
    mon = str(file)[-20:-18]
    day = str(file)[-18:-16] 
    YYYYMMDD_var3d.append(year + mon + day)
YYYYMMDD_var3d = YYYYMMDD_var3d[1:-1]  #chop off the vestigial 1st element.


# get days in addition to the months (YYYYMMDD_var2d). 
# I'm using the PW variable as an example and ASSUMING THAT IF THERE ARE ANY GAPS IN U, THERE ARE EQUIVALENT GAPS IN THE OTHER VARIABLES 

YYYYMMDD_var2d = ["00000000"]  
for file in PWfiles:
    year = str(file)[-24:-20]
    mon = str(file)[-20:-18]
    day = str(file)[-18:-16] 
    YYYYMMDD_var2d.append(year +  mon +  day)
YYYYMMDD_var2d = YYYYMMDD_var2d[1:-1] #chop off the vestigial 1st element.

days = YYYYMMDD_var3d

#print(days)

################################################################################
# loop thru each month label and find the 3D files that match that month:
################################################################################


#lat/lon bounds of sub-global domain: 
#note: .sel wasn't working properly on ERA5 data when lat1 < lat2. So, this results in -dy later, though proper sign of vertical voriticity, interstingly. I will correct
#the sign on dy later. 
lat1 = 70          #70  #67 #57.
lat2 = 10        #10 #20.
lon1 = 0 #10       #190 #0 190 #200.
lon2 = 360       #290 #360 294 #284.



#need to define a dummy array to concat things into:
era5fileU = xr.open_dataset(Ufiles[0])
dummy2dt = era5fileU.U.sel(longitude = slice(lon1,lon2), latitude  = slice(lat1,lat2)) #* units('m/s')
ddum,cdum,adum,bdum = dummy2dt.shape

ZS = range(cdum)
YS = range(adum)
XS = range(bdum)
TS = range(ddum)

#print("a,b 1",a,b)

longitude = copy.deepcopy(era5fileU.longitude)
latitude = copy.deepcopy(era5fileU.latitude)
longitude = longitude.sel(longitude = slice(lon1,lon2))
latitude = latitude.sel(latitude  = slice(lat1,lat2))
level = copy.deepcopy(era5fileU.level)
dx, dy = mpcalc.lat_lon_grid_deltas(longitude, latitude)

#### seed blank (x,y,t) arrays:
#times = np.array([], dtype='datetime64[ns]').reshape(0)
##basetime = np.array([], dtype=np.int64).reshape(0) 


######################################################################
# loop through dates to generate raw desired variables per day: 
######################################################################

##manually define target processing months (in commonmonths index space) I care about coresponding to the MCS climo - manual because python makes no sense:
#mjjas_2000thru2012 = [4,5,6,7,8,  16,17,18,19,20,  28,29,30,31,32,  40,41,42,43,44,   52,53,54,55,56,   64,65,66,67,68,   76,77,78,79,80, \
#                     88,89,90,91,92,  100,101,102,103,104,  112,113,114,115,116,   124,125,126,127,128,   136,137,138,139,140,    148,149,150,151,152]

#mjja_2000thru2012 = [4,5,6,7,  16,17,18,19,  28,29,30,31,  40,41,42,43,   52,53,54,55,   64,65,66,67,   76,77,78,79, \
#                     88,89,90,91,  100,101,102,103,  112,113,114,115,   124,125,126,127,   136,137,138,139,    148,149,150,151]

#mjjas_2011thru2012 = [136,137,138,139,140,    148,149,150,151,152]
#mjja_2012 = [148,149,150,151]
#mjja_2011 = [136,137,138,139]
#mjja_2009thru2010 = [112,113,114,115,   124,125,126,127]
#mjja_2006thru2008 = [76,77,78,79, 88,89,90,91,  100,101,102,103]
#mjja_2004thru2005 = [52,53,54,55,56,   64,65,66,67,68]
#mjjas_2011thru2012 = [136,137,138,139,140,    148,149,150,151,152]
#mjjas_2010thru2011 = [124,125,126,127,   136,137,138,139]


#mjja_2000thru2009 = [4,5,6,7,  16,17,18,19,  28,29,30,31,  40,41,42,43,   52,53,54,55,   64,65,66,67,   76,77,78,79, \
                     88,89,90,91,  100,101,102,103,  112,113,114,115]

#mjja_2016thru2021 = [196,197,198,199,  208,209,210,211,  220,221,222,223,  232,233,234,235, 244,245,246,247, 256,257,258,259]

mjja_2000thru2021 = [4,5,6,7,  16,17,18,19,  28,29,30,31,  40,41,42,43,   52,53,54,55,   64,65,66,67,   \
                     76,77,78,79,  88,89,90,91,  100,101,102,103,  112,113,114,115,   124,125,126,127,\
                     136,137,138,139,   148,149,150,151,  160,161,162,163,  172,173,174,175,  184,185,186,187,\
                     196,197,198,199,   208,209,210,211,  220,221,222,223,  232,233,234,235,  244,245,246,247,\
                     256,257,258,259]

#mjja_2000thru2015 = [4,5,6,7,  16,17,18,19,  28,29,30,31,  40,41,42,43,   52,53,54,55,   64,65,66,67,   \
#                     76,77,78,79,  88,89,90,91,  100,101,102,103,  112,113,114,115,   124,125,126,127,\
#                     136,137,138,139,   148,149,150,151,  160,161,162,163,  172,173,174,175,  184,185,186,187]

#mjja_2010thru2015 = [124,125,126,127,   136,137,138,139,    148,149,150,151,  160,161,162,163,  172,173,174,175,  184,185,186,187]
#mjja_2010thru2015 = [151,  160,161,162,163,  172,173,174,175,  184,185,186,187]


targetmonths =  mjja_2000thru2021

for mm in targetmonths:    
    mon = commonmonths[mm]
    
    print("MON in commonmonths ",mon)
    
    #load the monthly PW and sfc Pres files
    for spfile in SPfiles:
        if str(mon) in spfile:
            era5fileSP = xr.open_dataset(spfile)  
            spmon = era5fileSP.SP.sel(longitude = slice(lon1,lon2), latitude  = slice(lat1,lat2))     
    
    for pwfile in PWfiles:
        if str(mon) in pwfile:
            era5filePW = xr.open_dataset(pwfile)  
            pwmon = era5filePW.TCWV.sel(longitude = slice(lon1,lon2), latitude  = slice(lat1,lat2)) 
     
    for capefile in CAPEfiles:
        if str(mon) in capefile:
            era5fileCAPE = xr.open_dataset(capefile)  
            capemon = era5fileCAPE.CAPE.sel(longitude = slice(lon1,lon2), latitude  = slice(lat1,lat2)) 
            
    for viwvdfile in VIWVDfiles:
        if str(mon) in viwvdfile:
            era5fileVIWVD = xr.open_dataset(viwvdfile)  
            viwvdmon = era5fileVIWVD.VIWVD.sel(longitude = slice(lon1,lon2), latitude  = slice(lat1,lat2))

    for d in range(len(days)):
    #for d in range(4520,4521):    #test = [148]  #4520,4536
        
        day = days[d]

        
        if str(mon) in str(day[0:6]):      
            #day = days[d]
            
            print('days[d] ',d,' ', day)

            #### seed blank (x,y,t) arrays:
            times = np.array([], dtype='datetime64[ns]').reshape(0)

            #### seed raw wind(600,300), PW, vort600, PV
            U600   = np.array([], dtype=np.int64).reshape(0,adum,bdum)
            V600   = np.array([], dtype=np.int64).reshape(0,adum,bdum)
            W600   = np.array([], dtype=np.int64).reshape(0,adum,bdum)
            PV600   = np.array([], dtype=np.int64).reshape(0,adum,bdum)

            ##################################################
            #### now start the variable writing: 
            ##################################################

            #print(" DAY ",day)
            yyyy = day[0:4]
            mm = day[4:6]
            dd = day[6:8] 
            ymd = (str(yyyy) + "-" + str(mm) + "-" + str(dd) )

            pwday = pwmon.sel(time = str(ymd))     
            capeday = capemon.sel(time = str(ymd))  
            viwvdday = viwvdmon.sel(time = str(ymd))  
            spday = spmon.sel(time = str(ymd))

        
            
            ############################################################################
            # loop through files to generate raw desired variables from 3D files: 
            ############################################################################

            #daily U winds
            for ufile in Ufiles: 
                blahh = '.' + day + '00_'
                if blahh in str(ufile):
                    era5fileU = xr.open_dataset(ufile)   

                    #u3 = era5fileU.U.sel(longitude = slice(lon1,lon2), latitude  = slice(lat1,lat2), level = 300) #* units('m/s')
                    u6 = era5fileU.U.sel(longitude = slice(lon1,lon2), latitude  = slice(lat1,lat2), level = 600) #* units('m/s')  
                    U600 = np.append(U600, np.atleast_3d(u6), axis = 0)
                
                    times = np.append(times, u6.time)

                    # make array for secs since 1970 of each hour of this day:
                    secssince1970 = 999
                    for hr in TS:
                        #print(hr)
                        dd = day
                        d=datetime.datetime(int(dd[0:4]),int(dd[4:6]),int(dd[6:8]), hr)
                        secssince1970 = np.append(secssince1970,calendar.timegm(d.timetuple()))
                    secssince1970 = np.delete(secssince1970,0)


            #daily V winds:       
            for vfile in Vfiles: 
                if blahh in str(vfile): 
                    era5fileV = xr.open_dataset(vfile)   
                    v6 = era5fileV.V.sel(longitude = slice(lon1,lon2), latitude  = slice(lat1,lat2), level = 600) #* units('m/s')    
                    V600 = np.append(V600, np.atleast_3d(v6), axis = 0)    

                    
            #daily W winds:       
            for wfile in Wfiles: 
                if blahh in str(wfile): 
                    era5fileW = xr.open_dataset(wfile)   
                    w6 = era5fileW.W.sel(longitude = slice(lon1,lon2), latitude  = slice(lat1,lat2), level = 600) #* units('m/s')    
                    W600 = np.append(W600, np.atleast_3d(w6), axis = 0) 
                    
                    
            #daily PV winds:       
            for pvfile in PVfiles: 
                if blahh in str(pvfile):
                    era5filePV = xr.open_dataset(pvfile)   
                    pv6 = era5filePV.PV.sel(longitude = slice(lon1,lon2), latitude  = slice(lat1,lat2), level = 600) #* units('m/s')    
                    PV600 = np.append(PV600, np.atleast_3d(pv6), axis = 0)     


  
            #####################################
            #turn them into xarray for output: 
            #####################################         

            
            # # polish up vars
            ppw = pwday.values
            ccape = capeday.values
            vviwvd = viwvdday.values
            ssp = spday.values
            PW    = xr.DataArray(ppw, coords=[times, latitude, longitude], dims=["time", "latitude", "longitude"], name = "PW")
            CAPE  = xr.DataArray(ccape, coords=[times, latitude, longitude], dims=["time", "latitude", "longitude"], name = "CAPE")
            VIWVD = xr.DataArray(vviwvd, coords=[times, latitude, longitude], dims=["time", "latitude", "longitude"], name = "VIWVD")
            SP    = xr.DataArray(ssp, coords=[times, latitude, longitude], dims=["time", "latitude", "longitude"], name = "SP")        
         
            U600     = xr.DataArray(U600, coords=[times, latitude, longitude], dims=["time", "latitude", "longitude"],name = "U600")
            V600     = xr.DataArray(V600, coords=[times, latitude, longitude], dims=["time", "latitude", "longitude"],name = "V600")
            W600     = xr.DataArray(W600, coords=[times, latitude, longitude], dims=["time", "latitude", "longitude"],name = "W600") 
            PV600    = xr.DataArray(PV600, coords=[times, latitude, longitude], dims=["time", "latitude", "longitude"],name = "PV600")

            U600_bpf = copy.deepcopy(U600)
            U600_bpf = U600_bpf.transpose('time','longitude','latitude')
            V600_bpf = copy.deepcopy(V600)
            V600_bpf = V600_bpf.transpose('time','longitude','latitude')
            W600_bpf = copy.deepcopy(W600)
            W600_bpf = W600_bpf.transpose('time','longitude','latitude') 
            
            #print(CAPE)

            ##############################
            # create wavenumber filtered U & V to use throughout. 
            ##############################
            

            for ll in range(0,len(latitude)):   #lat loop
                
                wnmax = int( wn_range(latitude[ll],WL_min) )    #500 km  # yes, "wnmax" is intended to operate on "WL_min" (and viceversa)
                wnmin = int( wn_range(latitude[ll],WL_max) )   #2500 km
                
                utest = xr.DataArray(U600[:,ll:ll+1,:], coords=[times, latitude[ll:ll+1], longitude], dims=["time", "latitude", "longitude"])
                ufilt = wn_filter_3d( utest,  wnmin,  wnmax)
                U600_bpf[:,:,ll] = ufilt[:,0,:]
                
                vtest = xr.DataArray(V600[:,ll:ll+1,:], coords=[times, latitude[ll:ll+1], longitude], dims=["time", "latitude", "longitude"])
                vfilt = wn_filter_3d( vtest,  wnmin,  wnmax)
                V600_bpf[:,:,ll] = vfilt[:,0,:]

                wtest = xr.DataArray(W600[:,ll:ll+1,:], coords=[times, latitude[ll:ll+1], longitude], dims=["time", "latitude", "longitude"])
                wfilt = wn_filter_3d( wtest,  wnmin,  wnmax)
                W600_bpf[:,:,ll] = wfilt[:,0,:]    
                
                
            U600_bpf = U600_bpf.transpose('time','latitude','longitude')                
            V600_bpf = V600_bpf.transpose('time','latitude','longitude')   
            W600_bpf = W600_bpf.transpose('time','latitude','longitude') 
            
            U600_bpf = xr.DataArray(U600_bpf, coords=[times, latitude, longitude], dims=["time", "latitude", "longitude"],name = "U600_bpf")
            V600_bpf = xr.DataArray(V600_bpf, coords=[times, latitude, longitude], dims=["time", "latitude", "longitude"],name = "V600_bpf")   
            W600_bpf = xr.DataArray(W600_bpf, coords=[times, latitude, longitude], dims=["time", "latitude", "longitude"],name = "W600_bpf") 
  

            ############################################################
            #### DONE READING IN 2d/3d model fields. Now calculate derivative fields
            ############################################################

            #seed blank vorticity fields
            VOR600 = np.array([], dtype=np.int64).reshape(adum,bdum,0)  
            VOR600_bpf = np.array([], dtype=np.int64).reshape(adum,bdum,0)

            #seed blank stream function for the day
            SF600 = np.array([], dtype=np.int64).reshape(adum,bdum,0)
            SF600_bpf = np.array([], dtype=np.int64).reshape(adum,bdum,0)
            SFp600 = np.array([], dtype=np.int64).reshape(adum,bdum,0)
            SFp600_bpf = np.array([], dtype=np.int64).reshape(adum,bdum,0)


            #done grabbing the 3d variables, now processes them for this day, still in day loop:
            ttt = [0]

            for t in TS:

                ttt = np.append(ttt, str(times[t]))

                #calculate stream function (hourly over 3d space): 

                #convert to np format for windspharm use    
                u_np600 = U600[t,:,:].to_numpy()
                v_np600 = V600[t,:,:].to_numpy()
                u_np600_bpf = U600_bpf[t,:,:].to_numpy()
                v_np600_bpf = V600_bpf[t,:,:].to_numpy()

                #[if you need to chop out the sub-surface, probably do it here]

                ### Create an instance of the windspharm VectorWind class to do the computations.
                wf600 = VectorWind( u_np600, v_np600, gridtype = 'regular', rsphere=6371200.0 )
                wf600_bpf = VectorWind( u_np600_bpf, v_np600_bpf, gridtype = 'regular', rsphere=6371200.0 )

                ### Call methods to compute streamfunction and relative vorticity.
                psi6 = wf600.streamfunction()
                vor6 = wf600.vorticity()
                psi6_bpf = wf600_bpf.streamfunction()
                vor6_bpf = wf600_bpf.vorticity()

                SF600 = np.append(SF600, np.atleast_3d(psi6), axis = 2)   
                VOR600 = np.append(VOR600, np.atleast_3d(vor6), axis = 2) 
                SF600_bpf = np.append(SF600_bpf, np.atleast_3d(psi6_bpf), axis = 2)   
                VOR600_bpf = np.append(VOR600_bpf, np.atleast_3d(vor6_bpf), axis = 2)

                
            #now done with the day: 

            #"correct" dimension arrangemnt
            SF600 = SF600.transpose((2, 0, 1))
            VOR600 = VOR600.transpose((2, 0, 1))   
            SF600_bpf = SF600_bpf.transpose((2, 0, 1))
            VOR600_bpf = VOR600_bpf.transpose((2, 0, 1)) 

            # convert np arrays to xarrays:
            SF600 = xr.DataArray(SF600, coords=[times, latitude, longitude], dims=["time", "latitude", "longitude"],name = "SF600")          
            VOR600 = xr.DataArray(VOR600, coords=[times, latitude, longitude], dims=["time", "latitude", "longitude"],name = "VOR600")   
            PV600 = xr.DataArray(PV600, coords=[times, latitude, longitude], dims=["time", "latitude", "longitude"],name = "PV600")
            SF600_bpf = xr.DataArray(SF600_bpf, coords=[times, latitude, longitude], dims=["time", "latitude", "longitude"],name = "SF600_bpf")          
            VOR600_bpf = xr.DataArray(VOR600_bpf, coords=[times, latitude, longitude], dims=["time", "latitude", "longitude"],name = "VOR600_bpf") 


            #trim to subdomain:
                                     #global lon range       #oldies
            latA = 60       #70    #70       #70                     #70  #67 #57.
            latB = 20       #10    #10       #10                     #10 #20.
            lonA = 200     #190   #120        #0                      #190# 200.
            lonB = 300      #290   #320      #360                    #290 #360 294 #284.   
             
            pw = PW.sel(longitude = slice(lonA,lonB), latitude  = slice(latA,latB))
            cape = CAPE.sel(longitude = slice(lonA,lonB), latitude  = slice(latA,latB))
            viwvd = VIWVD.sel(longitude = slice(lonA,lonB), latitude  = slice(latA,latB))
            sp = SP.sel(longitude = slice(lonA,lonB), latitude  = slice(latA,latB))   
            
            u600 = U600.sel(longitude = slice(lonA,lonB), latitude  = slice(latA,latB))
            v600 = V600.sel(longitude = slice(lonA,lonB), latitude  = slice(latA,latB))
            w600 = W600.sel(longitude = slice(lonA,lonB), latitude  = slice(latA,latB))
            vor600 = VOR600.sel(longitude = slice(lonA,lonB), latitude  = slice(latA,latB))
            sf600 = SF600.sel(longitude = slice(lonA,lonB), latitude  = slice(latA,latB))   
            pv600 = PV600.sel(longitude = slice(lonA,lonB), latitude  = slice(latA,latB))
            
            u600_bpf = U600_bpf.sel(longitude = slice(lonA,lonB), latitude  = slice(latA,latB))
            v600_bpf = V600_bpf.sel(longitude = slice(lonA,lonB), latitude  = slice(latA,latB))
            w600_bpf = W600_bpf.sel(longitude = slice(lonA,lonB), latitude  = slice(latA,latB))
            vor600_bpf = VOR600_bpf.sel(longitude = slice(lonA,lonB), latitude  = slice(latA,latB))
            sf600_bpf = SF600_bpf.sel(longitude = slice(lonA,lonB), latitude  = slice(latA,latB))   

            PW = copy.deepcopy(pw)
            CAPE = copy.deepcopy(cape)
            VIWVD = copy.deepcopy(viwvd)
            SP = copy.deepcopy(sp)
            U600 = copy.deepcopy(u600)
            V600 = copy.deepcopy(v600)
            W600 = copy.deepcopy(w600)
            VOR600 = copy.deepcopy(vor600)
            SF600 = copy.deepcopy(sf600)               
            PV600 = copy.deepcopy(pv600)
  
            U600_bpf = copy.deepcopy(u600_bpf)
            V600_bpf = copy.deepcopy(v600_bpf)
            W600_bpf = copy.deepcopy(w600_bpf)
            VOR600_bpf = copy.deepcopy(vor600_bpf)
            SF600_bpf = copy.deepcopy(sf600_bpf) 

            #calculate stream fucntion anomalie on the subdomain at all times: 
            psi6pert = copy.deepcopy(SF600)
            aa,bb,cc = psi6pert.shape
            print(aa,bb,cc)            
            psi6pert = psi6pert.rename('SFp600')
            psimean = xr.DataArray.mean(psi6pert,dim='longitude')
            #print('psimean shape',psimean.shape)
            for la in range(bb):
                psi6pert[:,la,:] = SF600[:,la,:] - psimean[:,la]
                psi6pert[:,la,:] = -1 * psi6pert[:,la,:]
            SFp600 = psi6pert
            
            #calculate stream fucntion anomalie on the subdomain at all times: 
            psi6pert_bpf = copy.deepcopy(SF600_bpf)
            aa,bb,cc = psi6pert_bpf.shape
            print(aa,bb,cc)            
            psi6pert_bpf = psi6pert_bpf.rename('SFp600_bpf')
            psimean_bpf = xr.DataArray.mean(psi6pert_bpf,dim='longitude')
            for la in range(bb):
                psi6pert_bpf[:,la,:] = SF600_bpf[:,la,:] - psimean_bpf[:,la]
                psi6pert_bpf[:,la,:] = -1 * psi6pert_bpf[:,la,:]
            SFp600_bpf = psi6pert_bpf  
            

            ############################################
            #  do some mild simple #-pt 2D running-mean smoothing to iron out some kinks 
            ############################################

            aa,bb,cc = VOR600_bpf.shape
            print(aa,bb,cc)
            VOR600_bpf_sm1a = copy.deepcopy(VOR600_bpf)
            VOR600_bpf_sm1a[:,:,:] = 0.0
            VOR600_bpf_sm7pt = copy.deepcopy(VOR600_bpf)
            VOR600_bpf_sm7pt[:,:,:] = 0.0


            kernel_size1 = 7
            # kernel_size2 = 9
            # kernel_size3 = 13
            # kernel_size4 = 21

            for a in range(0,aa):       #time
                for b in range(0,bb):   #lat
                    #for c in range( 0+kernel_size2, cc-kernel_size2):   #lon
                    VOR600_bpf_sm1a[a,b,:] = np.convolve(VOR600_bpf[a,b,:], np.ones(kernel_size1) / kernel_size1, mode='same')

            for a in range(0,aa):       #time
                for c in range(0,cc):   #lat
                    VOR600_bpf_sm7pt[a,:,c] = np.convolve(VOR600_bpf_sm1a[a,:,c], np.ones(kernel_size1) / kernel_size1, mode='same')

            VOR600_bpf_sm7pt = xr.DataArray(VOR600_bpf_sm7pt, coords=[VOR600_bpf.time, VOR600_bpf.latitude, VOR600_bpf.longitude], dims=["time", "latitude", "longitude"],name = "VOR600_bpf_sm7pt")     

                
                
                

            #########################################
            #### write to netcdf output 
            #########################################

            filename = str(ymd)

            
            ttt = np.delete(ttt,0)
            ttt = xr.DataArray(ttt, coords=[times], dims=["time"],name = "Times")
            secssince1970 = xr.DataArray(secssince1970, coords=[times], dims=["time"],name = "basetime")
 
            print('writing netcdf ', filename + filename)

            # write the first var:
            PW.to_netcdf(fileout + filename + 'PertLonDom' + str(lonA) + '_' + str(lonB) + '.nc',mode='w')
            
            #now append more vars to file:
            
            #SP.to_netcdf(fileout + filename + 'PertLonDom' + str(lonA) + '_' + str(lonB) + '.nc',mode='a') 
            VIWVD.to_netcdf(fileout + filename + 'PertLonDom' + str(lonA) + '_' + str(lonB) + '.nc',mode='a')
            CAPE.to_netcdf(fileout + filename + 'PertLonDom' + str(lonA) + '_' + str(lonB) + '.nc',mode='a')
            
            ttt.to_netcdf(fileout + filename + 'PertLonDom' + str(lonA) + '_' + str(lonB) + '.nc', 'a')
            secssince1970.to_netcdf(fileout + filename + 'PertLonDom' + str(lonA) + '_' + str(lonB) + '.nc', 'a')
            
            U600.to_netcdf(fileout + filename + 'PertLonDom' + str(lonA) + '_' + str(lonB) + '.nc',  'a')
            V600.to_netcdf(fileout + filename + 'PertLonDom' + str(lonA) + '_' + str(lonB) + '.nc',  'a')   
            W600.to_netcdf(fileout + filename + 'PertLonDom' + str(lonA) + '_' + str(lonB) + '.nc',  'a')  
            U600_bpf.to_netcdf(fileout + filename + 'PertLonDom' + str(lonA) + '_' + str(lonB) + '.nc',  'a')
            V600_bpf.to_netcdf(fileout + filename + 'PertLonDom' + str(lonA) + '_' + str(lonB) + '.nc',  'a')  
            W600_bpf.to_netcdf(fileout + filename + 'PertLonDom' + str(lonA) + '_' + str(lonB) + '.nc',  'a') 
            
            VOR600.to_netcdf(fileout + filename + 'PertLonDom' + str(lonA) + '_' + str(lonB) + '.nc',  'a')          
            #VOR600_bpf.to_netcdf(fileout + filename + 'PertLonDom' + str(lonA) + '_' + str(lonB) + '.nc',  'a')  
            VOR600_bpf_sm7pt.to_netcdf(fileout + filename + 'PertLonDom' + str(lonA) + '_' + str(lonB) + '.nc',  'a')
            #VOR600_bpf_sm2b.to_netcdf(fileout + filename + 'PertLonDom' + str(lonA) + '_' + str(lonB) + '.nc',  'a') 
            
            #SFp600.to_netcdf(fileout + filename + 'PertLonDom' + str(lonA) + '_' + str(lonB) + '.nc',  'a')
            SFp600_bpf.to_netcdf(fileout + filename + 'PertLonDom' + str(lonA) + '_' + str(lonB) + '.nc',  'a')       
       
            
   